In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2014, 2015)

# processor = FootballDataProcessor('../Data/Serie_A_2014_games.json')
# final_df = processor.process(data_folder, file_names, years)
# final_df.rename(columns={'player_id': 'player'}, inplace=True)


# # Garantir que ambas as colunas 'player' estejam no tipo string
# final_df['player'] = final_df['player'].astype('string')
# id_players['player'] = id_players['player'].astype('string')

# # Realizar o merge sem utilizar índices, mas mantendo a coluna 'player'
# final_df_ = pd.merge(final_df, id_players, on='player', how='left')

# # Definir os nomes dos times que deseja analisar
# team_1_name = 'Criciúma / SC'  # Substitua pelo nome real do time 1 
# team_2_name = 'Palmeiras / SP'  # Substitua pelo nome real do time 2

# # Criar uma instância de FootballMatchRating
# rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

# # Atualizar as ratings
# updated_match_data = rating_calculator.update_ratings()

# # Garantir que 'player' em updated_match_data está como string
# updated_match_data['player'] = updated_match_data['player'].astype('string')

# # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
# updated_values = id_players[['player']].merge(
#     updated_match_data[['player', 'rating', 'games_played']],
#     on='player',
#     how='left'
# )

# # Atualizar apenas as linhas correspondentes em id_players
# id_players.update(updated_values)

In [4]:
df_jogos_2014 = pd.read_json('../Data/Serie_A_2014_games.json')
df_jogos_2014_ = df_jogos_2014.T

for index, row in df_jogos_2014_.iterrows():
    processor = FootballDataProcessor(df_jogos_2014_, index-1)
    final_df = processor.process(data_folder, file_names, years)
    final_df.rename(columns={'player_id': 'player'}, inplace=True)

    final_df['player'] = final_df['player'].astype('string')
    id_players['player'] = id_players['player'].astype('string')

    final_df_ = pd.merge(final_df, id_players, on='player', how='left')

    team_1_name = df_jogos_2014_.iloc[index-1,0]
    team_2_name = df_jogos_2014_.iloc[index-1,1]

    # Criar uma instância de FootballMatchRating
    rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

    # Atualizar as ratings
    updated_match_data = rating_calculator.update_ratings()
    # Garantir que 'player' em updated_match_data está como string
    updated_match_data['player'] = updated_match_data['player'].astype('string')
    # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
    updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
    updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

    updated_values = id_players[['player']].merge(
        updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
        on='player',
        how='left'
    )

    # Atualizar apenas as linhas corespondentes em id_players
    id_players.update(updated_values)

id_players


,player,rating,age,games_played,player_name,time_jogador
0,149237,1512.536316,25,58,7Nilmar Nilmar Honorato da Silva,Internacional / RS
1,331937,1458.863224,25,88,20Lucas Lucas Rafael Araujo Lima,Santos / SP
2,176729,1514.148571,25,70,25Hudson Hudson Rodrigues dos ...,São Paulo / SP
3,308980,1483.150667,25,74,29Alison Alison Lopes Ferreira,Santos / SP
4,171547,1514.381747,25,68,8Gil Jose Gildeixon Cleme...,Coritiba / PR
...,...,...,...,...,...,...
731,185679,1501.821466,25,86,4Cicinho Neuciano de Jesus Gusmao,Santos / SP
732,291380,1523.463710,25,65,13Paulo Mira ... Jonathan Doin,São Paulo / SP
733,145298,1646.169492,25,68,13Wendel Wendel Santana Perei ...,Palmeiras / SP
734,174348,1468.308881,25,84,44Leandro Al ... Leandro Almeida da Silva,Coritiba / PR


In [3]:
x = id_players.sort_values(by='rating')
x.to_excel('teste.xlsx')